In [1]:
import os, sys

sys.path.append(os.path.join(os.getcwd(), '../src/'))
from data_reader.data_reader import DataReader

In [8]:
dr = DataReader()
shot_df = dr.read_shots("shots-20201201010000-data", start_shot_number=1, end_shot_number=10)
shot_df.head()

,timestamp,sequential_number,sequential_number_by_shot,displacement,load01,load02,load03,load04,shot_number,tags
0,1.606752e+09,0,0,49.171,0.568421,0.526316,0.597368,0.502632,1,"[異常A, 異常B]"
1,1.606752e+09,1,1,49.245,0.550000,0.526316,0.623684,0.507895,1,"[異常A, 異常B]"
2,1.606752e+09,2,2,49.136,0.568421,0.510526,0.660526,0.521053,1,"[異常A, 異常B]"
3,1.606752e+09,3,3,49.184,0.600000,0.502632,0.678947,0.521053,1,"[異常A, 異常B]"
4,1.606752e+09,4,4,49.149,0.578947,0.473684,0.671053,0.521053,1,"[異常A, 異常B]"


In [9]:
shots_meta_df = dr.read_shots_meta("shots-20201201010000-meta")
shots_meta_df.head()

,shot_number,spm,num_of_samples_in_cut_out
0,1,80.598572,3125.0
1,2,80.613753,3113.0
2,3,80.594235,3129.0
3,4,80.636509,3097.0
4,5,80.582338,3140.0


In [10]:
sys.path.append(os.path.join(os.getcwd(), '../src/analyze/'))
from analyze.h_one_extract_features import *
# from analyze.analyze import *
# analyze.main(target="20201201010000")

In [11]:
shot_df

,timestamp,sequential_number,sequential_number_by_shot,displacement,load01,load02,load03,load04,shot_number,tags
0,1.606752e+09,0,0,49.171,0.568421,0.526316,0.597368,0.502632,1,"[異常A, 異常B]"
1,1.606752e+09,1,1,49.245,0.550000,0.526316,0.623684,0.507895,1,"[異常A, 異常B]"
2,1.606752e+09,2,2,49.136,0.568421,0.510526,0.660526,0.521053,1,"[異常A, 異常B]"
3,1.606752e+09,3,3,49.184,0.600000,0.502632,0.678947,0.521053,1,"[異常A, 異常B]"
4,1.606752e+09,4,4,49.149,0.578947,0.473684,0.671053,0.521053,1,"[異常A, 異常B]"
...,...,...,...,...,...,...,...,...,...,...
28042,1.606752e+09,28042,3108,34.055,1.202632,0.828947,0.360526,0.415789,9,"[異常A, 異常B]"
28043,1.606752e+09,28043,3109,34.055,1.144737,0.786842,0.300000,0.405263,9,"[異常A, 異常B]"
28044,1.606752e+09,28044,3110,34.055,1.110526,0.781579,0.252632,0.384211,9,"[異常A, 異常B]"
28045,1.606752e+09,28045,3111,34.055,1.057895,0.757895,0.213158,0.350000,9,"[異常A, 異常B]"


In [14]:
for shot in range(1, 10):
    shot_data = shot_df[shot_df.shot_number == shot]

#     argmax,valmax = extract_features(shot_data, 80.0, breaking_varmax29, shot=shot, disp_chart=True)
    argmax,valmax = extract_features(shot_data, 80.0, breaking_varmax29, shot=shot)
    print(argmax, valmax)

[2197, 2208, 2198, 2198] [5.644736842105263, 0.7499999999999991, -1.6315789473684146, -0.6289473684210495]
[2226, 2226, 2233, 2233] [0.007894736842117833, 1.1499999999999906, -0.39999999999999836, 0.5421052631578975]
[2193, 2193, 2192, 2192] [-1.1421052631578876, 0.349999999999993, 3.392105263157898, 3.2315789473684164]
[2143, 2142, 2143, 2142] [5.118421052631582, 4.157894736842092, 0.24999999999998995, 1.300000000000013]
[2220, 2220, 2219, 2219] [1.0342105263158166, 1.2789473684210453, 1.1315789473684366, 1.881578947368419]
[2213, 2214, 2216, 2216] [4.773684210526312, 3.505263157894744, -0.7526315789473628, 0.26842105263159177]
[2164, 2161, 2164, 2164] [2.7263157894736594, 4.465789473684214, -1.8605263157894634, -0.7052631578947475]
[2197, 2208, 2198, 2198] [5.644736842105263, 0.7499999999999991, -1.6315789473684146, -0.6289473684210495]
[2226, 2226, 2233, 2233] [0.007894736842117833, 1.1499999999999906, -0.39999999999999836, 0.5421052631578975]
